# enviorment (package)

In [65]:
%matplotlib inline
import csv
import pandas as pd
import numpy as np
import networkx as nx
import mygene

# material process

In [66]:
physical_network = r'D:/downloads/9606.protein.links.v10.txt/9606.protein.links.v10.txt'
network = r'D:/downloads/9606.protein.links.v10.txt/9606.protein.links.v10.txt'
data = pd.read_csv(network,sep=" ")
print (data)
print(type(data))

                     protein1              protein2  combined_score
0        9606.ENSP00000000233  9606.ENSP00000003084             150
1        9606.ENSP00000000233  9606.ENSP00000003100             215
2        9606.ENSP00000000233  9606.ENSP00000005257             223
3        9606.ENSP00000000233  9606.ENSP00000005340             193
4        9606.ENSP00000000233  9606.ENSP00000006101             415
...                       ...                   ...             ...
8547997  9606.ENSP00000473243  9606.ENSP00000467958             150
8547998  9606.ENSP00000473243  9606.ENSP00000468280             204
8547999  9606.ENSP00000473243  9606.ENSP00000468367             251
8548000  9606.ENSP00000473243  9606.ENSP00000470087             254
8548001  9606.ENSP00000473243  9606.ENSP00000471417             153

[8548002 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>


In [67]:
data["protein1"] = data['protein1'].str.lstrip("9609.")

data["protein2"] = data['protein2'].str.lstrip("9609.")

data['combined_score'] = data["combined_score"]/1000


In [68]:
type(data['combined_score'][1])
data.to_csv(r'../data/STRINGv10Cut9606.txt', header=['#protein1','protein2','combined_score'], index=None, sep=' ', mode='w')
#如果沒有在protein1前面加上#的話，會讓下一步的讀檔出問題


In [69]:
#check dataframe to graph，如果設定graph()的話，好像會吃掉一些edge
#所以要設定成digraph
#但不知道為什麼，沒有辦法加上權重
#透過先寫檔再讀檔的方式來解決問題

network_file = open(r'../data/STRINGv10Cut9606.txt','rb')
G = nx.read_weighted_edgelist(network_file,comments='#',create_using=nx.DiGraph())

#G = nx.from_pandas_edgelist(data,'protein1','protein2',edge_attr='combined_score',create_using=nx.DiGraph()) #一直沒辦法加入權重的寫法

In [70]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 19247
Number of edges: 8548002
Average in degree: 444.1213
Average out degree: 444.1213


In [71]:
print(nx.is_weighted(G))

True


In [72]:
print(nx.is_weighted(G,('ENSP00000000233','ENSP00000379496')))

NetworkXError: Edge ('ENSP00000000233', 'ENSP00000379496') does not exist.

In [73]:
peptideID = data["protein1"].append(data["protein2"]).unique()

In [74]:
print(len(peptideID))
peptideID

19247


array(['ENSP00000000233', 'ENSP00000000412', 'ENSP00000000442', ...,
       'ENSP00000473200', 'ENSP00000473233', 'ENSP00000473243'],
      dtype=object)

## high  confidence? to nature paper ?300w edge?

# tranfer peptideID to geneID

In [75]:
mg = mygene.MyGeneInfo()
results_string_physical= mg.querymany(peptideID, scopes='ensembl.protein',fields = 'symbol',species='human',returnall=True,as_dataframe=True,df_index=True,verbose=True)
#19000個輸入，然後有608個找不到transfer gene id，但是這邊的數字，怎麼會比high confidence score的部分還要少...
#del peptideID

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-19247...done.
Finished.
1686 input query terms found no hit:
	['ENSP00000004921', 'ENSP00000005905', 'ENSP00000006101', 'ENSP00000035383', 'ENSP00000053469', 'ENS


In [76]:
print(results_string_physical['out']['symbol'])

query
ENSP00000000233       ARF5
ENSP00000000412       M6PR
ENSP00000000442      ESRRA
ENSP00000001008      FKBP4
ENSP00000001146    CYP26B1
                    ...   
ENSP00000473172     RUVBL2
ENSP00000473193        NaN
ENSP00000473200        NaN
ENSP00000473233        NaN
ENSP00000473243        NaN
Name: symbol, Length: 19247, dtype: object


In [77]:
print(results_string_physical)

{'out':                              _id     _score   symbol notfound
query                                                        
ENSP00000000233              381  28.461329     ARF5      NaN
ENSP00000000412             4074  28.466175     M6PR      NaN
ENSP00000000442             2101  27.552513    ESRRA      NaN
ENSP00000001008             2288  27.556425    FKBP4      NaN
ENSP00000001146            56603  28.468815  CYP26B1      NaN
...                          ...        ...      ...      ...
ENSP00000473172            10856  28.468815   RUVBL2      NaN
ENSP00000473193              NaN        NaN      NaN     True
ENSP00000473200  ENSG00000282246  28.467943      NaN      NaN
ENSP00000473233  ENSG00000142539  28.463629      NaN      NaN
ENSP00000473243              NaN        NaN      NaN     True

[19247 rows x 4 columns], 'dup': Empty DataFrame
Columns: [query, duplicate hits]
Index: [], 'missing':                 query
0     ENSP00000004921
1     ENSP00000005905
2     ENSP00000

In [78]:
string_physical_IDTransfer = results_string_physical['out'].reset_index()

In [79]:
#string_physical_IDTransfer = string_physical_IDTransfer.rename(mapper={'ensembl.gene':'ensembl_gene'},axis=1)

In [80]:
#get whole ensembl gene id 
string_physical_gene_id = string_physical_IDTransfer['symbol'].unique()
print(len(string_physical_gene_id))

17356


In [81]:
string_physical_ensembl_gene_list = string_physical_IDTransfer[['query','symbol']]
print(string_physical_ensembl_gene_list)

                 query   symbol
0      ENSP00000000233     ARF5
1      ENSP00000000412     M6PR
2      ENSP00000000442    ESRRA
3      ENSP00000001008    FKBP4
4      ENSP00000001146  CYP26B1
...                ...      ...
19242  ENSP00000473172   RUVBL2
19243  ENSP00000473193      NaN
19244  ENSP00000473200      NaN
19245  ENSP00000473233      NaN
19246  ENSP00000473243      NaN

[19247 rows x 2 columns]


In [82]:
#relabel node
print(type(results_string_physical['missing']['query']))
print(results_string_physical['missing'])

<class 'pandas.core.series.Series'>
                query
0     ENSP00000004921
1     ENSP00000005905
2     ENSP00000006101
3     ENSP00000035383
4     ENSP00000053469
...               ...
1681  ENSP00000472951
1682  ENSP00000473005
1683  ENSP00000473092
1684  ENSP00000473193
1685  ENSP00000473243

[1686 rows x 1 columns]


In [83]:
string_physical_network = G.copy()
#string_physical_ensembl_gene_list = string_physical_ensembl_gene_list.query('ensembl_gene' != 'NaN')
print(nx.info(string_physical_network))


Name: 
Type: DiGraph
Number of nodes: 19247
Number of edges: 8548002
Average in degree: 444.1213
Average out degree: 444.1213


In [84]:
#for i in range(len(results_string_physical['missing'])):
    #print(results_string_physical['missing'].iloc[i])
missing = results_string_physical['missing']['query'].tolist()
for i in missing:
    string_physical_network.remove_node(i)
print(nx.info(string_physical_network))

Name: 
Type: DiGraph
Number of nodes: 17561
Number of edges: 7395152
Average in degree: 421.1122
Average out degree: 421.1122


In [85]:
print(nx.is_weighted(string_physical_network))

True


# rename the dataframe

先建構成網路，然後再把前面對應到的字詞抽出來，然後mapping上去

In [ ]:
print(string_physical_ensembl_gene_list)

In [ ]:
string_physical_ensembl_gene_list = string_physical_ensembl_gene_list[['query','symbol']]
print(string_physical_ensembl_gene_list)

In [ ]:
string_physical_ensembl_gene_list['symbol'] = string_physical_ensembl_gene_list['symbol'].map(np.nan_to_num)
print(string_physical_ensembl_gene_list)

In [ ]:
i = string_physical_ensembl_gene_list[string_physical_ensembl_gene_list.symbol == 0 ].index
print(len(i))

In [ ]:
string_physical_ensembl_gene_list = string_physical_ensembl_gene_list.drop(i)
print(string_physical_ensembl_gene_list)

In [ ]:
string_physical_ensembl_gene_dict = string_physical_ensembl_gene_list.set_index('query').T.to_dict('record')
print(string_physical_ensembl_gene_dict)

In [ ]:
print(type(string_physical_ensembl_gene_dict))
string_physical_ensembl_gene_dict = string_physical_ensembl_gene_dict[0]
print(type(string_physical_ensembl_gene_dict))

In [ ]:
print(type(string_physical_network))
string_physical_network = nx.relabel_nodes(string_physical_network,string_physical_ensembl_gene_dict)

In [ ]:
###需要修改#####
node = list(nx.nodes(string_physical_network))
a = node.copy()
for i in a:
    if i.startswith("ENSP"):
        string_physical_network.remove_node(i)

In [ ]:
print(nx.info(string_physical_network))
string_physical_network_node_list= string_physical_network.nodes()
print(len(string_physical_network_node_list))

In [45]:
#write STRING-geneID network file
nx.readwrite.edgelist.write_weighted_edgelist(G=string_physical_network,path='../data/STRING10_network_process_GeneSymbol.txt',delimiter='\t')

# Construct Network

In [63]:
print(nx.info(string_physical_network))

Name: 
Type: DiGraph
Number of nodes: 17351
Number of edges: 7238712
Average in degree: 417.1928
Average out degree: 417.1928


In [64]:
string_physical_network.remove_nodes_from(list(nx.isolates(string_physical_network)))
print(nx.info(string_physical_network))

Name: 
Type: DiGraph
Number of nodes: 17351
Number of edges: 7238712
Average in degree: 417.1928
Average out degree: 417.1928


In [58]:
string_physical_network_undirected = string_physical_network.to_undirected()
largest_cc_test = max(nx.connected_components(string_physical_network_undirected), key=len)
print(len(largest_cc_test))

17351


In [60]:
node_not_in_large_cc = []
for i in node:
    if i not in largest_cc_test:
        node_not_in_large_cc.append(i)
for k in node:
    if k in node_not_in_large_cc:
        string_physical_network.remove_node(k)
print(nx.info(network_test))

NetworkXError: The node ENSP00000347689 is not in the digraph.

In [61]:
network_test = string_physical_network.copy()
for (u, v, wt) in string_physical_network.edges.data('weight'):
    if wt < 0.7:
        network_test.remove_edge(u, v)
print(nx.info(network_test))

Name: 
Type: DiGraph
Number of nodes: 17351
Number of edges: 508930
Average in degree:  29.3315
Average out degree:  29.3315


In [62]:
network_test.remove_nodes_from(list(nx.isolates(network_test)))
print(nx.info(network_test))

Name: 
Type: DiGraph
Number of nodes: 14147
Number of edges: 508930
Average in degree:  35.9744
Average out degree:  35.9744
